In [1]:
import pandas as pd

In [6]:
lyrics = pd.read_csv('../genius_lyrics.csv')

In [19]:
lyrics['0'][7] 

'Song of the Year“Formation”— Beyoncé\n“Hello” — Adele\n“I Took A Pill In Ibiza” — Mike Posner\n“Love Yourself” —Justin Bieber\n“7 Years” — Lukas GrahamRecord of the Year“Hello”— Adele\n“Formation” — Beyoncé\n“7 Years” — Lukas Graham\n“Work” — Rihanna Featuring Drake\n“Stressed Out” — Twenty One PilotsAlbum of the Year25 — Adele\nLemonade — Beyoncé\nPurpose — Justin Bieber\nViews — Drake\nA Sailor’s Guide To Earth — Sturgill SimpsonBest New ArtistKelsea Ballerini\nThe Chainsmokers\nChance The Rapper\nMaren Morris\nAnderson .PaakBest Rap/Sung Performance:“Freedom” — Beyoncé Featuring Kendrick Lamar\n“Hotline Bling” — Drake\n“Broccoli” — D.R.A.M. Featuring Lil Yachty\n“Ultralight Beam” — Kanye West Featuring Chance The Rapper, Kelly Price, Kirk Franklin & The-Dream\n“Famous” — Kanye West Featuring RihannaBest Rap Performance:“All The Way Up” — Fat Joe & Remy Ma Featuring French Montana & Infared\n"Panda" — Desiigner\n“No Problem” — Chance The Rapper Featuring Lil Wayne & 2 Chainz\n“Pop S

What to clean out: 
    * \n
    * anything after a —

### I want to see how wordy/lengthy these songs are
Could possibly run it with length of song to see just how much message is getting packed into there.<br>
Message = not repetitive, lots of words per ms?

In [13]:
#Character count
char_count = []

for i in range(0,496):
    chars = len(lyrics['0'][i])
    char_count.append(chars) 

In [14]:
# word count
def count_words(string):
    words = string.split()
    return len(words)